In [126]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Make multiple prints work
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [127]:
tmc = pd.read_json('tmc.json')

tmc.drop(['volume_usd'], inplace=True, axis=1)

tmc['date'] = tmc['market_cap_by_available_supply'].astype(str).str.extract('(?P<date>\d{4}-\d{2}-\d{2})')
tmc['date'] = pd.to_datetime(tmc['date'], format='%Y-%m-%d')

tmc['tmc'] = tmc['market_cap_by_available_supply'].astype(str).str.extract('(?P<value>\d+\.\d)', expand=False)
tmc['tmc'] = tmc['tmc'].astype(float).astype(int)

tmc.drop(['market_cap_by_available_supply'], inplace=True, axis=1)

tmc.set_index('date', inplace=True)
# Remove the duplicated for some reason last record
tmc.drop(tmc.index[len(tmc)-1], inplace=True)

# Check for duplicates
tmc[tmc.index.duplicated()]

df = tmc

,tmc
date,


In [128]:
def setup(df, coinName):
    
    df.drop(['price_btc', 'volume_usd'], inplace=True, axis=1)
    
    df['date'] = df['price_usd'].astype(str).str.extract('(?P<date>\d{4}-\d{2}-\d{2})')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df[coinName + '_price'] = df['price_usd'].astype(str).str.extract('(?P<price>\d+\.\d+)').astype(float)
    df[coinName + '_mc'] = df['market_cap_by_available_supply'].astype(str).str.extract(', (?P<price>\d+)').astype(int)
    df.drop(['market_cap_by_available_supply'], inplace=True, axis=1)
    df.drop(['price_usd'], inplace=True, axis=1)
    
    df.set_index('date', inplace=True)
    # Remove the duplicated for some reason last record
    df.drop(df.index[len(df)-1], inplace=True)
    
    # Check for duplicates
    # print(df[df.index.duplicated()])
    
    return df

df['tmc_ret'] = np.log(df['tmc'].pct_change(1)+1)

for coinName in ['btc', 'eth']: # , 'xrp', 'xlm', 'ltc'
    df = df.join(setup(pd.read_json(coinName + '.json'), coinName))
    df[coinName + '_ret'] = np.log(df[coinName + '_price'].pct_change(1)+1)

df.head()

,tmc,tmc_ret,btc_price,btc_mc,btc_ret,eth_price,eth_mc,eth_ret
date,,,,,,,,
2016-01-01,7098250000,NaN,431.196,6.480617e+09,NaN,0.933442,70849438.0,NaN
2016-01-02,7139190000,0.005751,433.818,6.521456e+09,0.006062,0.948646,72026659.0,0.016157
2016-01-03,7144890000,0.000798,433.842,6.523378e+09,0.000055,0.940705,71447659.0,-0.008406
2016-01-04,7067980000,-0.010823,428.833,6.449670e+09,-0.011613,0.968536,73586334.0,0.029156
2016-01-05,7151150000,0.011698,434.155,6.531656e+09,0.012334,0.957776,72793217.0,-0.011172


In [129]:
def calc_event_stats(ev_name, ev_date, win_est, win_evnt):
    
    """
    Calculates how significant the impact of an event on coins' prices was.

    :param date_ev: the date of the event
    :param win_est: the length of the estimation window
    :param win_evnt: the one-side length of the event window (day-2,day-1,event,day+1,day+2)
    :returns: 
    :raises keyError: raises an exception if there is no historical data for a specific range
    """
    
    print('=== Event: ', ev_name, ' ===')
    
    date_ev = pd.to_datetime(ev_date, format='%Y-%m-%d')
    est_l1 = date_ev - pd.to_timedelta(win_est + win_evnt, unit='d')
    est_l2 = date_ev - pd.to_timedelta(win_evnt + 1, unit='d')
    eve_l1 = date_ev - pd.to_timedelta(win_evnt, unit='d')
    eve_l2 = date_ev + pd.to_timedelta(win_evnt, unit='d')
    
    print('Esimation window:', est_l1.strftime('%Y-%m-%d'), ':', est_l2.strftime('%Y-%m-%d'))
    print('Event window:', eve_l1.strftime('%Y-%m-%d'), ':', eve_l2.strftime('%Y-%m-%d'))
    print('Event date:', date_ev.strftime('%Y-%m-%d'))

    est_w = df.loc[est_l1:est_l2].copy()
    eve_w = df.loc[eve_l1:eve_l2].copy()
    
    # Calculate for total market cap
    
#     ret = 'tmc_ret'
#     ar = 'tmc_ar'
#     t = 'tmc_t'
#     car = 'tmc_car'
    
#     tmcret = df['tmc_ret'].dropna()
#     model = sm.OLS(est_w[ret], tmcret[est_l1:est_l2]).fit()
#     slope = model.params.values[0]
#     se = model.bse.values[0]
#     print(slope, se)

#     eve_w[ar] = eve_w[ret] - (tmcret[eve_l1:eve_l2] * slope)
#     eve_w[t] = eve_w[ar] / se
#     eve_w[car] = eve_w[ar].cumsum()
    
    for coinName in ['btc', 'eth']: # , 'xrp', 'xlm', 'ltc'
        mc = coinName + '_mc'
        ret = coinName + '_ret'
        ar = coinName + '_ar'
        t = coinName + '_t'
        car = coinName + '_car'

        tmcret = df['tmc'] - df[mc]
        tmcret = np.log(tmcret.pct_change(1)+1)
        tmcret = tmcret.dropna()
        model = sm.OLS(est_w[ret], tmcret[est_l1:est_l2]).fit()
        #model.summary()
        slope = model.params.values[0]
        se = model.bse.values[0]

        eve_w[ar] = eve_w[ret] - (tmcret[eve_l1:eve_l2] * slope)
        eve_w[t] = eve_w[ar] / se
        eve_w[car] = eve_w[ar].cumsum()
        eve_w = eve_w.round(2)
    
    s = eve_w.filter(regex='(_ret$)|(_ar$)|(_t$)|(_car$)').style.apply(lambda x: ["background: lightgreen" if v > 1.96 else "background: red" if v < -1.96 else "" for v in x], axis = 1)
    s.index = s.index.normalize()
    return s

ew = 120

calc_event_stats('DAO attack', '2016-06-17', ew, 2)
calc_event_stats('Bitfinex attack', '2016-08-02', ew, 2)
calc_event_stats('Parity multi-sig bug', '2017-11-07', ew, 2)
calc_event_stats('NiceHash attack', '2017-12-06', ew, 2)
calc_event_stats('Coincheck attack', '2018-01-26', ew, 2)
calc_event_stats('BitGrail attack', '2018-02-09', ew, 2)

calc_event_stats('Consensus summit', '2017-05-23', ew, 3)
calc_event_stats('Bitcoin Cash hardfork', '2017-08-01', ew, 2)
calc_event_stats('CME Group BTC futures', '2017-10-31', ew, 2)
calc_event_stats('CBOE BTC futures', '2017-12-01', ew, 2)
calc_event_stats('Bitconnect closing', '2018-01-16', ew, 2)
calc_event_stats('Twitter ban', '2018-03-26', ew, 2)

calc_event_stats('Brexit', '2016-06-24', ew, 7)
calc_event_stats('Trump election', '2016-11-08', ew, 7)
calc_event_stats('Trump tariffs', '2018-03-01', ew, 7)


=== Event:  DAO attack  ===
Esimation window: 2016-02-16 : 2016-06-14
Event window: 2016-06-15 : 2016-06-19
Event date: 2016-06-17


,tmc_ret,btc_ret,eth_ret,btc_ar,btc_t,btc_car,eth_ar,eth_t,eth_car
date,,,,,,,,,
2016-06-15 00:00:00,-0.01,-0.02,0.01,-0.02,-0.96,-0.02,0.01,0.02,0.01
2016-06-16 00:00:00,0.01,0.08,0.02,0.07,3.08,0.05,0.02,0.06,0.03
2016-06-17 00:00:00,0.1,0.01,0.08,0.03,1.14,0.07,0.1,0.31,0.13
2016-06-18 00:00:00,-0.07,0.02,-0.4,0,0.17,0.08,-0.41,-1.21,-0.27
2016-06-19 00:00:00,-0.01,-0,-0.08,-0,-0.19,0.07,-0.08,-0.24,-0.36


=== Event:  Bitfinex attack  ===
Esimation window: 2016-04-02 : 2016-07-30
Event window: 2016-07-31 : 2016-08-04
Event date: 2016-08-02


,tmc_ret,btc_ret,eth_ret,btc_ar,btc_t,btc_car,eth_ar,eth_t,eth_car
date,,,,,,,,,
2016-07-31 00:00:00,-0,-0.03,-0.02,-0.02,-1.29,-0.02,-0.02,-0.09,-0.02
2016-08-01 00:00:00,-0.03,-0.02,-0.06,-0.03,-1.36,-0.05,-0.06,-0.24,-0.08
2016-08-02 00:00:00,-0.03,-0.03,-0.12,-0.03,-1.72,-0.08,-0.12,-0.51,-0.19
2016-08-03 00:00:00,-0.13,-0.09,-0.08,-0.11,-5.78,-0.2,-0.06,-0.27,-0.26
2016-08-04 00:00:00,0.06,0.04,0.09,0.05,2.63,-0.14,0.08,0.36,-0.17


=== Event:  Parity multi-sig bug  ===
Esimation window: 2017-07-08 : 2017-11-04
Event window: 2017-11-05 : 2017-11-09
Event date: 2017-11-07


...

,tmc_ret,btc_ret,eth_ret,btc_ar,btc_t,btc_car,eth_ar,eth_t,eth_car
date,,,,,,,,,
2016-06-17 00:00:00,0.1,0.01,0.08,0.04,1.82,0.04,0.1,0.29,0.1
2016-06-18 00:00:00,-0.07,0.02,-0.4,-0.01,-0.4,0.03,-0.41,-1.23,-0.31
2016-06-19 00:00:00,-0.01,-0,-0.08,-0.01,-0.26,0.03,-0.08,-0.24,-0.39
2016-06-20 00:00:00,0.02,0.01,-0.09,0.02,0.78,0.04,-0.09,-0.26,-0.48
2016-06-21 00:00:00,-0.03,-0.07,-0.01,-0.06,-2.66,-0.02,-0.02,-0.05,-0.49
2016-06-22 00:00:00,-0.1,-0.05,0.25,-0.07,-3.32,-0.09,0.23,0.7,-0.26
2016-06-23 00:00:00,-0.07,-0.14,-0.14,-0.12,-5.59,-0.21,-0.15,-0.46,-0.41
2016-06-24 00:00:00,0.04,0.1,0.09,0.09,3.95,-0.12,0.1,0.29,-0.32
2016-06-25 00:00:00,0.05,0.04,0.01,0.04,2.04,-0.08,0.02,0.05,-0.3


=== Event:  Trump election  ===
Esimation window: 2016-07-04 : 2016-10-31
Event window: 2016-11-01 : 2016-11-15
Event date: 2016-11-08


,tmc_ret,btc_ret,eth_ret,btc_ar,btc_t,btc_car,eth_ar,eth_t,eth_car
date,,,,,,,,,
2016-11-01 00:00:00,0,0.03,-0.06,0.04,2,0.04,-0.06,-0.37,-0.06
2016-11-02 00:00:00,0.03,-0,0.01,-0,-0.23,0.03,0,0.01,-0.06
2016-11-03 00:00:00,0.01,0.02,0.01,0.02,0.94,0.05,0.01,0.05,-0.05
2016-11-04 00:00:00,-0.07,-0.04,0.01,-0.04,-2.02,0.01,0.03,0.2,-0.02
2016-11-05 00:00:00,0.03,-0,-0.01,-0.01,-0.32,0.01,-0.02,-0.13,-0.04
2016-11-06 00:00:00,0,0.01,-0,0.01,0.49,0.02,-0,-0.01,-0.04
2016-11-07 00:00:00,0.01,-0.01,-0.01,-0.02,-0.8,0,-0.01,-0.08,-0.06
2016-11-08 00:00:00,-0.01,0,-0,0,0.22,0.01,0,0.02,-0.05
2016-11-09 00:00:00,0.01,0.04,-0.03,0.04,2.19,0.05,-0.03,-0.2,-0.09


=== Event:  Trump tariffs  ===
Esimation window: 2017-10-25 : 2018-02-21
Event window: 2018-02-22 : 2018-03-08
Event date: 2018-03-01


,tmc_ret,btc_ret,eth_ret,btc_ar,btc_t,btc_car,eth_ar,eth_t,eth_car
date,,,,,,,,,
2018-02-22 00:00:00,-0.07,-0.01,-0.03,-0.01,-0.12,-0.01,-0.02,-0.25,-0.02
2018-02-23 00:00:00,-0.06,-0.09,0.04,-0.09,-1.33,-0.1,0.05,0.46,0.02
2018-02-24 00:00:00,0.03,0.06,-0.03,0.06,0.8,-0.05,-0.03,-0.34,-0.01
2018-02-25 00:00:00,-0.05,-0.07,0.01,-0.07,-0.96,-0.11,0.01,0.14,0
2018-02-26 00:00:00,0.01,-0.02,0.03,-0.02,-0.28,-0.13,0.03,0.3,0.03
2018-02-27 00:00:00,0.04,0.07,0.02,0.07,0.98,-0.06,0.02,0.17,0.05
2018-02-28 00:00:00,0.02,0.06,-0.01,0.06,0.84,-0,-0.01,-0.12,0.04
2018-03-01 00:00:00,-0.04,-0.06,0.01,-0.06,-0.79,-0.06,0.01,0.13,0.05
2018-03-02 00:00:00,0.03,0.06,-0.03,0.06,0.92,0,-0.03,-0.34,0.02
